In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from torch.utils.data import DataLoader
from hcmus.core import appconfig
from hcmus.utils import viz_utils
from hcmus.lbs import LabelStudioConnector
from hcmus.data import CroppedImageTextDatasetV2

2025-06-29 17:01:03.613 | INFO     | hcmus.core.appconfig:<module>:7 - Load DotEnv: True
/Volumes/Cucumber/Projects/item-identification/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
import mlflow
experiment_name = "/CLIP"
experiment = mlflow.get_experiment_by_name(experiment_name)
experiment_id = None
if not experiment:
    experiment_id = mlflow.create_experiment(experiment_name)
else:
    experiment_id = experiment.experiment_id

In [4]:
accepted_labels = """
8934804026817-nuoc-tuong-maggi-dau-nanh-thanh-diu-chai-700ml
8934804020402-nuoc-tuong-maggi-dau-nanh-dam-dac-700ml
8936048470524-mi-tom-3-mien-gold-vi-bo-ham-rau-thom-75g
8934822201272-bia-heineken-silver-lon-250ml
5054563022592-kem-danh-rang-sensodyne-giam-e-buotlam-sang-rang-100g
8934868166344-nuoc-rua-tay-lifebouy-450ml
8934707029182-hat-nem-knorr-thit-than-xuong-ong-va-tuy-900g
8934822361211-bia-heineken-khong-con-lon-330ml
8850006932346-kem-danh-rang-colgate-maxfresh-huong-tra-xanh-225g-+-ban-chai-long-to
8850006932322-kem-danh-rang-colgate-maxfresh-huong-bac-ha-225g-+-ban-chai-long-to
6920354827792-kem-danh-rang-colgate-maxfresh-tinh-chat-than-tre-225g-+-ban-chai-long-to
8936136163314-nuoc-mam-nam-ngu-3-trong-1-750ml
8934563198152-mi-hao-hao-chay-vi-rau-nam-74gr
8935049501503-nuoc-ngot-coca-cola-vi-nguyen-ban-lon-320ml
8936048470036-mi-3-mien-mi-chay-la-da-65g-(goi)
8935001215028-tra-tea-break-milk-350ml
8935006532755-kem-rua-mat-acnes-vitamin-lam-sang-da,-mo-seo-tham-50g
8935018801641-khan-bep-da-nang-pulppy-classic-2-lop-loc-2-cuon
8935024123287-ca-phe-sua-hoa-tan-g7-3in1-hop-336g-(16g-x-21-goi)
8935006538634-dau-tay-trang-hada-labo-sach-sau-duong-am-toi-uu-200ml
8934840000093-nuoc-khoang-chanh-lemona-chai-500ml
8997035601321-nuoc-uong-bu-dien-giai-bo-sung-ion-pocari-sweat-chai-900ml
4897036693162-nuoc-tang-luc-monster-mango-loco-355ml
809939073648-soda-sua-huong-vi-chuoi-milkis-banana
8934804038131-tra-hoa-qua-nestea-hoa-tan-hop-144g-12-goi-x-12g
8936048471248-mi-tom-chua-cay-dac-biet-3-mien-gold-goi-75g
8934804027333-tra-chanh-nestea-hoa-tan-hop-195g-15-goi-x-13g
8936010530881-tra-chanh-cozy-hoa-tan-hop-240g-16-goi-x-15g
8935049510604-nuoc-ngot-sprite-chai-320ml
8934588672118-nuoc-ngot-pepsi-khong-calo-vi-chanh-320ml
8935049501572-nuoc-ngot-fanta-vi-cam-lon-320ml
8935049500445-nuoc-ngot-fanta-chai-1.5l
8936721790017-nuoc-loc-th-true-water-chai-500ml
8935049501718-nuoc-ngot-sprite-lon-320ml
8936193070013-tra-xanh-khong-do-500ml
8935137601122-tra-xanh-thai-nguyen-100g
8934564600883-tra-den-c2-vi-dua-luoi-chai-455ml
8935001214274-tra-bi-dao-wonderfarm-lon-310ml
8934588870552-tra-olong-teaplus-450ml
6920354836930-kem-danh-rang-colgate-optic-white-cong-nghe-sac-tim-100g
8935049501374-nuoc-ngot-cocacola-vi-nguyen-ban-chai-600ml
8935039570700-ca-phe-sua-robusta-birdy-lon-170ml
8934673701402-nuoc-dao-necta-vinamilk-hop-1l
8935005801012-nuoc-loc-lavie-chai-1-5l
8936079140021-ca-phe-sua-highland-coffee-lon-185ml
8934588640445-ca-phe-sua-boss-lon-180ml
8934683008614-ca-phe-sua-vinacafe-3-in-1-gold-tui-480g-24-goi-tui
8934683008867-nuoc-tang-luc-wake-up-247-vi-ca-phe-330ml
8935328600156-tra-xanh-tui-loc-phuc-long-2g-x-hop-25-goi
4902430805322-nuoc-rua-tay-diet-khuan-safeguard-trang-tinh-khiet-450ml
8936156730718-nuoc-xa-vai-blue-dam-dac-huong-thanh-xuan-tui-3.2l
8934868166825-dau-goi-sunsilk-natural-duong-ngan-gay-rung-650g
8934804040523-tra-vai-va-huong-hoa-lai-nestea-hoa-tan-hop-144g-12-goi-x-12g
8934822801335-bia-tiger-crystal-4-6-phan-tram-lon-330ml
8934868156871-dau-xa-sunsilk-mem-muot-dieu-ky-320g
8936010530713-tra-dao-cozy-hoa-tan-hop-16-goi-x-15g
8934868171850-nuoc-lau-kinh-sunlight-sieu-nhanh-sach-trong-suot-chai-520ml
8850228003541-nuoc-tang-luc-warrior-huong-dau-chai-330ml
8936193070075-nuoc-tang-luc-number-1-chai-330ml
8934588012112-nuoc-ngot-pepsi-cola-320ml
8936013254951-nuoc-rua-kinh-gift-540ml
8934868190394-nuoc-giat-comfort-huong-hoa-anh-dao-va-dao-tuoi-tui-3kg
3068320055008-nuoc-loc-evivan-chai-500ml
8936127794206-nuoc-uong-sua-trai-cay-th-true-juice-milk-huong-viet-quat-300ml
8935049511038-sua-trai-cay-nutriboost-cam-chai-1l
8934673573399-sua-tuoi-tiet-trung-co-duong,-100%-sua-tuoivinamilk-(1lit).
3760128640607-sua-tuoi-tiet-trung-promess-nguyen-kem-hop-1l
8934822212339-bia-heineken-lon-cao-330ml
8801382123446-nuoc-gao-han-quoc-loai-500ml
8936036201604-bia-hoegaarden-rosee-chai-248ml
8934692090013-bia-1664-blanc-5%-abv-chai-330ml
8934868166870-dau-goi-sunsilk-ong-muot-rang-ngoi-chai-650g
8934588233074-nuoc-tang-luc-sting-vi-dau-tay-do-330ml
8934822431211-bia-tiger-soju-infused-lager-wonder-melon-vi-dua-luoi-lon-330ml
8934822112332-bia-tiger-lon-cao-330ml-lon
8934868180470-dau-goi-dove-biotin-ngan-gay-rung-toc-880g
8935024170519-ca-phe-trung-nguyen-legend-classic-hop-357g-17g-x-21-goi
8934868166351-nuoc-rua-tay-lifebuoy-matcha-va-lo-hoichai-450g
8936191270057-ca-phe-cappuccino-dua-just-viet-hop-hop-10-goi-x-17g
8934868166924-dau-goi-sunsilk-mem-muot-dieu-ky-chai-650g
8934804028064-ca-phe-hoa-tan-nescafe-3in1-vi-nguyen-ban-tui-782g-(17g-x-46-goi)
8936122200276-ca-phe-mr.brown-blue-mountain-240ml
8934563122201-mi-hao-hao-vi-tom-chua-cay-goi-100g
8935136865709-sua-tam-purite-duong-da-mem-min-huong-hoa-hong-rose-chai-850ml
8934822451295-nuoc-trai-cay-len-men-strongbow-vi-thom-luu-3.5%-lon-330ml
8934868170990-nuoc-rua-tay-lifebuoy-vitamin-sua-duong-am-tui-1kg
8936094291005-bia-budweiser-5-lon-330ml
8934822121297-nuoc-trai-cay-len-men-strongbow-red-beries-4.5%-abv-lon-320ml
8934822111298-nuoc-trai-cay-len-men-strongbow-gold-apple-4-5-abv-lon-320ml
8935049501190-nuoc-tra-fuze-tea-vi-chanh-sa-450ml
8936199810026-cafe-the-coffee-house-sua-da-22gx10-goi-(hop)
8888589308920-tra-bi-dao-winter-melon-jj-lon-300ml
8934673101097-sua-tuoi-tiet-trung-tiet-trung-vinamilk-khong-duong-bich-220ml
8934563183158-mi-hao-hao-sa-te-hanh-tim-75g
8936048470012-mi-3-mien-tom-chua-cay-65g
8934683009925-ca-phe-sua-da-vinacafe-chat-hop-10-goi-24g
8850228007617-nuoc-tang-luc-redbull-250ml
8934563651138-mi-hao-hao,-vi-tom-chua-cayacecook,-coc-(67g)
8934588063060-nuoc-loc-aquafina-chai-1-5l
8935049510857-nuoc-loc-dasani-chai-1.5l"""
accepted_labels = accepted_labels.splitlines()
accepted_labels = [x for x in accepted_labels if len(x) > 0]

In [5]:
from hcmus.utils import data_utils
from hcmus.utils import transform_utils
splits = data_utils.get_data_splits()

2025-06-29 17:01:05.572 | WARNING  | hcmus.lbs._label_studio_connector:get_tasks:148 - Page size is too large, only 3443 tasks available.
2025-06-29 17:01:05.572 | INFO     | hcmus.lbs._label_studio_connector:get_tasks:152 - New `page_to` applied: 35
Loading tasks: 100%|██████████| 35/35 [00:09<00:00,  3.60it/s]
2025-06-29 17:01:20.906 | WARNING  | hcmus.lbs._label_studio_connector:get_tasks:148 - Page size is too large, only 375 tasks available.
2025-06-29 17:01:20.907 | INFO     | hcmus.lbs._label_studio_connector:get_tasks:152 - New `page_to` applied: 4
Loading tasks: 100%|██████████| 4/4 [00:02<00:00,  1.45it/s]
2025-06-29 17:01:25.467 | WARNING  | hcmus.lbs._label_studio_connector:get_tasks:148 - Page size is too large, only 245 tasks available.
2025-06-29 17:01:25.467 | INFO     | hcmus.lbs._label_studio_connector:get_tasks:152 - New `page_to` applied: 3
Loading tasks: 100%|██████████| 3/3 [00:01<00:00,  2.40it/s]


In [6]:
splits["train"][1]

{'image': '/Volumes/Cucumber/Projects/datasets/raw/hcmus-iid-lbs/7bcaf33a6048f4038b1e509eef754014.jpeg',
 'task': LabelStudioTask(id=2559, data={'image': '/data/upload/1/3f869ef1-IMG_4369.jpeg'}, meta={}, created_at=datetime.datetime(2025, 6, 1, 2, 5, 39, 830584, tzinfo=TzInfo(UTC)), updated_at=datetime.datetime(2025, 6, 27, 1, 18, 37, 975364, tzinfo=TzInfo(UTC)), is_labeled=True, overlap=1, inner_id=1795, total_annotations=1, cancelled_annotations=0, total_predictions=0, comment_count=0, unresolved_comment_count=0, last_comment_updated_at=None, project=1, updated_by=1, file_upload=3970, comment_authors=[], annotations=[Annotation(id=2363, result=[AnnotationResult(original_width=800, original_height=600, image_rotation=0, value=RectangleLabel(x=2.865612648221343, y=44.40052700922269, width=97.13438735177854, height=27.404479578392653, rotation=0.0, rectanglelabels=['8934804020402-nuoc-tuong-maggi-dau-nanh-dam-dac-700ml']), id='Vaq5AfTUra', from_name='label', to_name='image', type='rect

In [7]:
print("Validate labels and boxes")
for split_name in splits.keys():
    for idx, item in enumerate(splits[split_name]):
        a = item.get("target").get("labels")
        b = item.get("target").get("boxes")
        assert len(a) == len(b), f"{len(a)} vs {len(b)}, Split={split_name} item at {idx}"
print("PASSED!")

Validate labels and boxes
PASSED!


In [8]:
transform_train, transform_test = transform_utils.get_transforms_downscale_random_clip()
datasets = data_utils.get_image_datasets_v2(splits, transform_train, transform_test)

2025-06-29 17:01:27.920 | INFO     | hcmus.data._torch_dataset_v2:__init__:51 - Apply random_margin=0.3
2025-06-29 17:01:28.097 | INFO     | hcmus.data._torch_dataset_v2:__init__:62 - Auto infer `label2idx` mapping, mapping length: 99.
2025-06-29 17:01:28.101 | INFO     | hcmus.data._torch_dataset_v2:__init__:51 - Apply random_margin=0.0
2025-06-29 17:01:28.425 | INFO     | hcmus.data._torch_dataset_v2:__init__:51 - Apply random_margin=0.0
2025-06-29 17:01:28.507 | INFO     | hcmus.data._torch_dataset_v2:__init__:51 - Apply random_margin=0.0


### Finetune model

In [9]:
import torch
import clip
from hcmus.models.clip import CLIPFineTuner
from hcmus.models.clip import FinetuneMode
from hcmus.const.product import PRODUCT_DESCRIPTION

device = "cuda" if torch.cuda.is_available() else "cpu"
finetuner = CLIPFineTuner(device=device)

skip_labels = [
    "object",
    "8936079140021-ca-phe-sua-highland-coffee-lon-185ml",
]

train_dataset = CroppedImageTextDatasetV2(
    splits["train"],
    product_desc=PRODUCT_DESCRIPTION,
    skip_labels=skip_labels,
    preprocess_fn=finetuner.preprocess,
    tokenizer=clip.tokenize,
    random_margin=0.2
)
val_dataset = CroppedImageTextDatasetV2(
    splits["val"],
    product_desc=PRODUCT_DESCRIPTION,
    skip_labels=skip_labels,
    label2idx=train_dataset.label2idx,
    preprocess_fn=finetuner.preprocess,
    tokenizer=clip.tokenize
)
val_dataset = CroppedImageTextDatasetV2(
    splits["test"],
    product_desc=PRODUCT_DESCRIPTION,
    skip_labels=skip_labels,
    label2idx=train_dataset.label2idx,
    preprocess_fn=finetuner.preprocess,
    tokenizer=clip.tokenize
)

2025-06-29 17:01:30.383 | INFO     | hcmus.data._torch_dataset_v2:__init__:51 - Apply random_margin=0.2
2025-06-29 17:01:30.551 | INFO     | hcmus.data._torch_dataset_v2:__init__:62 - Auto infer `label2idx` mapping, mapping length: 98.
2025-06-29 17:01:30.555 | INFO     | hcmus.data._torch_dataset_v2:__init__:51 - Apply random_margin=0.0


Loaded CLIP model: ViT-B/32
Total parameters: 151,277,313


2025-06-29 17:01:30.640 | INFO     | hcmus.data._torch_dataset_v2:__init__:51 - Apply random_margin=0.0


In [ ]:
finetuner.finetune(
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    finetune_mode=FinetuneMode.PARTIAL_TOP,
    top_layers=3,  # Train top 3 layers
    epochs=200,
    lr=5e-5,
    batch_size=12,
    experiment_id=experiment_id
)


Finetuning Mode: partial_top
Trainable parameters: 31,376,129 (20.74%)
Frozen parameters: 119,901,184

Trainable parameters:
  text_projection: 262,144 params
  logit_scale: 1 params
  visual.proj: 393,216 params
  visual.transformer.resblocks.9.attn.in_proj_weight: 1,769,472 params
  visual.transformer.resblocks.9.attn.in_proj_bias: 2,304 params
  visual.transformer.resblocks.9.attn.out_proj.weight: 589,824 params
  visual.transformer.resblocks.9.attn.out_proj.bias: 768 params
  visual.transformer.resblocks.9.ln_1.weight: 768 params
  visual.transformer.resblocks.9.ln_1.bias: 768 params
  visual.transformer.resblocks.9.mlp.c_fc.weight: 2,359,296 params
  visual.transformer.resblocks.9.mlp.c_fc.bias: 3,072 params
  visual.transformer.resblocks.9.mlp.c_proj.weight: 2,359,296 params
  visual.transformer.resblocks.9.mlp.c_proj.bias: 768 params
  visual.transformer.resblocks.9.ln_2.weight: 768 params
  visual.transformer.resblocks.9.ln_2.bias: 768 params
  visual.transformer.resblocks.10.

Epoch 1:   0%|          | 0/220 [00:00<?, ?it/s]/Volumes/Cucumber/Projects/item-identification/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Volumes/Cucumber/Projects/item-identification/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Volumes/Cucumber/Projects/item-identification/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Volumes/Cucumber/Projects/item-identification/.venv/lib/python3

Train Loss: 2.1229


Validation:   0%|          | 0/462 [00:00<?, ?it/s]/Volumes/Cucumber/Projects/item-identification/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Volumes/Cucumber/Projects/item-identification/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Volumes/Cucumber/Projects/item-identification/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Volumes/Cucumber/Projects/item-identification/.venv/lib/pyth

Val Loss: 3.7132, Val Acc: 0.1692


...

Epoch 40:   0%|          | 0/220 [00:00<?, ?it/s]/Volumes/Cucumber/Projects/item-identification/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Volumes/Cucumber/Projects/item-identification/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Volumes/Cucumber/Projects/item-identification/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Volumes/Cucumber/Projects/item-identification/.venv/lib/python

Train Loss: 0.1032


Validation:   0%|          | 0/462 [00:00<?, ?it/s]/Volumes/Cucumber/Projects/item-identification/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Volumes/Cucumber/Projects/item-identification/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Volumes/Cucumber/Projects/item-identification/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Volumes/Cucumber/Projects/item-identification/.venv/lib/pyth

Val Loss: 2.7930, Val Acc: 0.1885

Epoch 41/200


Epoch 41:   0%|          | 0/220 [00:00<?, ?it/s]/Volumes/Cucumber/Projects/item-identification/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Volumes/Cucumber/Projects/item-identification/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Volumes/Cucumber/Projects/item-identification/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Volumes/Cucumber/Projects/item-identification/.venv/lib/python